# Training Module

In [1]:
import os
os.chdir('genre_classification_289a/src')

In [2]:
import torch
import torch.nn as nn
import os
from model import STN, MLP
import torch.optim as optim

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
from features import get_data_loaders, FramedFeatureDataset, FeatureDataset, DatasetSettings

### DCNN Training

In [15]:
#STN targets
agfs = [] #'subgenre', 'mfcc'
genre = True #False if not genre STN

target = None

if (genre and len(agfs) > 0) or (len(agfs) > 0):
    raise 'Error: can only train one STN target at a time'
elif genre:
    target = 'genre'
elif len(agfs) == 1:
    target = agfs[0]
else:
    raise 'Error: could not parse training target'

print('Training STN target: {}'.format(target))
    
#dataset
dataset_name = 'fma_medium'

Training STN target: genre


In [16]:
settings = DatasetSettings(dataset_name, 'fma_metadata')
dataset = FramedFeatureDataset(settings,  agfs=agfs, genre=genre)
print("Num genres: ", settings.num_genres)
print(settings.genre_counts)

Num genres:  16
Rock                   6911
Electronic             6110
Experimental           2207
Hip-Hop                2109
Folk                   1477
Instrumental           1280
Pop                    1129
International          1004
Classical               598
Old-Time / Historic     510
Jazz                    380
Country                 178
Soul-RnB                154
Spoken                  118
Blues                    72
Easy Listening           21
Name: genre_top, dtype: int64


In [17]:
output_size = settings.num_genres
stn = STN(output_size)
stn.to(device)
stn = nn.DataParallel(stn)

In [18]:
## Training Parameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(stn.parameters(), lr=0.001)
epochs = 30
batch_size = 64
valid_split = 0.2

In [19]:
trainloader, validloader = get_data_loaders(dataset, batch_size, valid_split)

In [20]:
import torch.nn.functional as F
from sklearn.metrics import f1_score

def validate(stn, label_name):
    
    with torch.no_grad():
        stn.eval()
        
        all_pred = []
        all_true = []
        
        for i, data in enumerate(validloader, 0):
            inputs, labels = data[0].to(device), data[1][label_name].to(device)
            
            out = stn(inputs)
            loss = F.cross_entropy(out, labels)
            
            all_pred.append(out.argmax(dim=1))
            all_true.append(labels)
            
        all_pred = torch.cat(all_pred)
        all_true = torch.cat(all_true)
        
        curr_f1 = f1_score(all_true.cpu(), all_pred.cpu(), average='micro')
        
        print('Validation f1 score: {}'.format(curr_f1))

In [11]:
#Train it
for epoch in range(epochs):  # loop over the dataset multiple times
    
    print('Starting epoch', epoch + 1)
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data[0].to(device), data[1]['genre'].to(device)  #data[1]['{argument for agf being trained}']
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = stn(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 30 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 30))
            running_loss = 0.0
            
    if epoch % 5 == 4:
        validate(stn, 'genre')
        stn.train()

print('Finished Training')

Starting epoch 1
[1,    50] loss: 3.094
[1,   100] loss: 2.833
[1,   150] loss: 2.625
[1,   200] loss: 2.513
[1,   250] loss: 2.495
[1,   300] loss: 2.404
[1,   350] loss: 2.383
[1,   400] loss: 2.371
[1,   450] loss: 2.312
[1,   500] loss: 2.233
[1,   550] loss: 2.233
[1,   600] loss: 2.272
[1,   650] loss: 2.246
[1,   700] loss: 2.193
Starting epoch 2
[2,    50] loss: 2.182
[2,   100] loss: 2.166
[2,   150] loss: 2.104
[2,   200] loss: 2.071
[2,   250] loss: 2.077
[2,   300] loss: 2.051
[2,   350] loss: 2.062
[2,   400] loss: 2.073
[2,   450] loss: 2.027
[2,   500] loss: 2.021
[2,   550] loss: 1.971
[2,   600] loss: 2.034
[2,   650] loss: 1.967
[2,   700] loss: 1.973
Starting epoch 3
[3,    50] loss: 1.968
[3,   100] loss: 1.934
[3,   150] loss: 1.886
[3,   200] loss: 1.842
[3,   250] loss: 1.880
[3,   300] loss: 1.947
[3,   350] loss: 1.837
[3,   400] loss: 1.874
[3,   450] loss: 1.837
[3,   500] loss: 1.866
[3,   550] loss: 1.860
[3,   600] loss: 1.842
[3,   650] loss: 1.854
[3,   

### Save Model

In [12]:
model_file = '../models/DCNN_{}_{}'.format(dataset_name, target)
torch.save(stn, model_file)

## Load & Eval Model

In [22]:
model_file = '../models/no-seed-model-backups/DCNN_{}_{}'.format('fma_small', 'mfcc')
model = torch.load(model_file)
model

RuntimeError: cuda runtime error (710) : device-side assert triggered at /opt/conda/conda-bld/pytorch_1579022060824/work/torch/csrc/generic/serialization.cpp:148

In [21]:
validate(model, target)

RuntimeError: CUDA error: device-side assert triggered

## Previous STN Val Accuracies
FMA_SMALL
* genre: 89.X% (can't reload model to validate b/c didn't set a random seed during training and the random split of training / validation data is different so we're mixing training data into the validation set and getting  unrealistically high validation accuracies)

FMA_MED
* genre: 89.3%
